In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def extract_min_vertex_from_queue(distances, queue):
    dist_for_queue =  {k:distances[k] for k in queue}
    return [k for k,v in sorted(dist_for_queue.items(), key= lambda x: x[1])][0]

def dijkstra(M, start_node, end_node):
    # initialize parameters
    coordinates = M.intersections       # dictionary  {node index : [x,y] location coordinates}
    distances = {node: np.inf for node in M._graph.nodes()}
    distances[start_node] = 0
    queue = M._graph.nodes()
    
    while queue != []:
        #print(extract_min_vertex_from_queue(distances, queue).value)
        current_node = extract_min_vertex_from_queue(distances, queue)
        queue.remove(current_node)
        for neighbour in M.roads[current_node]:
            dist = distances[current_node] + calculate_distance(current_node, neighbour, coordinates)
            if dist < distances[neighbour]:
                distances[neighbour] = dist
    
    return distances[end_node]

In [3]:
def calculate_distance(nodeA, nodeB, coordinates):
    xa,ya = coordinates[nodeA]
    xb,yb = coordinates[nodeB]
    return np.sqrt( (xa-xb)**2 + (ya-yb)**2 )

In [4]:
def find_nearest_frontier_node(routes):
    path_costs =  {node:routes[node][1] for node in routes}
    return [node for node,path_cost in sorted(path_costs.items(), key= lambda x: x[1])][0]

In [5]:
def shortest_path_Uniform_Cost_Search(M,start,goal):
   
    
    print("shortest path called")
    print('Minimum traversing distance to reach goal using Dijkstra greedy algorithm: {:.2f}'.format(dijkstra(M, start, goal)))
   
    #nodes = M._graph.nodes()           
    coordinates = M.intersections       
    neighbours = M.roads               
    explored = set()                    
    frontier = {start:([start], 0)}      
                                        
    candidates = []                     
    while len(frontier) > 0:
        current_node = find_nearest_frontier_node(frontier)
        current_route, path_cost = frontier[current_node]
        frontier.pop(current_node)
        explored.add(current_node)
        
        
        for neighbour in neighbours[current_node]:
            new_route = deepcopy(current_route)
            new_route.append(neighbour)
            step_cost = calculate_distance(current_node, neighbour, coordinates)
            new_path_cost = path_cost + step_cost

            if neighbour == goal:
                candidates.append((new_route, new_path_cost))
            elif neighbour not in explored:
                if neighbour not in frontier:
                    frontier[neighbour] = (new_route, new_path_cost) 
                else:
                    existing_cost = frontier[neighbour][1]
                    if new_path_cost < existing_cost:
                        frontier[neighbour] = (new_route, new_path_cost)
    shortest_route = None
    shortest_distance = None
    for candidate, path_cost in candidates:
        if not shortest_route:
            shortest_route = candidate
            shortest_distance = path_cost
        elif path_cost < shortest_distance:
            shortest_route = candidate
            shortest_distance = path_cost
    for candidate in candidates:
        print(candidate)
    print('Minimum traversing distance to reach goal using Uniform Cost greedy algorithm: {:.2f}'.format(shortest_distance))
    return shortest_route

In [6]:
def goaltest(location, goal):
    return location == goal
def find_nearest_frontier_node_AStar(routes, goal, coordinates):
    path_costs =  {node:routes[node][1] + calculate_distance(node, goal, coordinates) for node in routes}
    return [node for node, path_cost in sorted(path_costs.items(), key= lambda x: x[1])][0]

In [7]:
def shortest_path(M,start,goal):
    
    print("shortest path called")
    print('Minimum traversing distance to reach goal using Dijkstra greedy algorithm: {:.2f}'.format(dijkstra(M, start, goal)))
    #nodes = M._graph.nodes()           # list of all intersections of the graph M
    coordinates = M.intersections       
    neighbours = M.roads                
    explored = set()                    
    frontier = {start:([start], 0)}      
    while len(frontier) > 0:

        current_node = find_nearest_frontier_node_AStar(frontier, goal, coordinates)

        current_route, path_cost = frontier[current_node]

        if goaltest(current_node, goal):
            break
        frontier.pop(current_node)

        explored.add(current_node)
        for neighbour in neighbours[current_node]:
            new_route = deepcopy(current_route)
            new_route.append(neighbour)
            step_cost = calculate_distance(current_node, neighbour, coordinates)
            new_path_cost = path_cost + step_cost
            if neighbour not in explored:
                if neighbour not in frontier:
                    frontier[neighbour] = (new_route, new_path_cost) 
                else:
                    existing_cost = frontier[neighbour][1]
                    if new_path_cost < existing_cost:
                        frontier[neighbour] = (new_route, new_path_cost)
    print('Minimum traversing distance to reach goal using A* algorithm: {:.2f}'.format(frontier[goal][1]))
    return current_route

In [8]:
def print_best_path(j, Q, start, goal):
    print('shortest path and distance to target:')
    sum_costs = 0
    current_node = start
    while current_node != goal:
        print(current_node,'->',end=' ')
        # Move to the next node and increment costs
        next_node = np.argmin(Q[current_node, :] + j)
        sum_costs += Q[current_node, next_node]
        current_node = next_node
    print(goal)
    print('Cost: {:.04f}'.format(sum_costs))
        
def shortest_path_DP(M,start,goal):
    print("shortest path called")
    print('Minimum traversing distance to reach goal using Dijkstra greedy algorithm: {:.2f}'.format(dijkstra(M, start, goal)))
    nodes = M._graph.nodes()               
    coordinates = M.intersections          
    neighbours = M.roads                   
    j = np.zeros_like(nodes, dtype='float')
    next_j = np.empty_like(nodes, dtype='float')
    Q = np.ones((len(nodes),len(nodes)))
    Q = Q * np.inf                         
    for node in nodes:
        for neighbour in neighbours[node]:
            if Q[node, neighbour] == np.inf:
                   Q[node, neighbour] = calculate_distance(node, neighbour, coordinates)
    Q[goal, goal] = 0
    max_iter = 500
    i=0
    while i < max_iter:
        for nodeA in nodes:
            next_j[nodeA] = np.min(Q[nodeA,:] + j)   # Bellman equation
        #if np.equal(j, next_j).all():   # for integers
        if np.allclose(next_j, j):       # to use with floats
            print('iterations converged after',i,'steps with dynamic programming')
            break
        else:
            j[:] = next_j    # copy contents of next_j to j
            i+=1
    print_best_path(j, Q, start, goal) 
    return Q,j, j[start]